# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [5]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [7]:
import pandas as pd
import numpy as np

# Load Energy dataset from Excel file
energy = pd.read_excel('assets/Energy Indicators.xls', header=17, skipfooter=38)
energy = energy.iloc[:, 2:] # remove first two columns
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy.replace('...', np.NaN, inplace=True) # replace missing data with NaN
energy['Energy Supply'] *= 1000000 # convert to gigajoules
energy['Country'] = energy['Country'].str.replace(r'\(.*\)', '').str.strip() # remove parenthesis from country names
energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}, inplace=True)

# Load GDP dataset from CSV file
GDP = pd.read_csv('assets/world_bank.csv', header=4)
GDP['Country Name'].replace({
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}, inplace=True)
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
GDP = GDP[['Country'] + list(GDP.columns[-10:])]

# Load ScimEn dataset from Excel file
ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')

# Join the three datasets
df = pd.merge(ScimEn, energy, on='Country', how='inner')
df = pd.merge(df, GDP, on='Country', how='inner')
df.set_index('Country', inplace=True)
df = df[df['Rank'] <= 15] # select top 15 countries by rank
df = df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] + list(df.columns[-10:])]
df.sort_values(by='Rank', inplace=True)

# Return resulting dataframe
def answer_one():
    return df

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12


In [9]:
import pandas as pd
import numpy as np

def answer_one():
    # load energy data
    energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38, usecols=[2,3,4,5], names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], na_values='...')
    energy['Energy Supply'] *= 1000000
    
    # clean country names
    energy['Country'] = energy['Country'].str.replace(r"\(.*\)","").str.strip()
    energy['Country'].replace({"Republic of Korea": "South Korea",
                               "United States of America": "United States",
                               "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                               "China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)
    
    # load GDP data
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    GDP['Country Name'].replace({"Korea, Rep.": "South Korea", 
                                 "Iran, Islamic Rep.": "Iran",
                                 "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
    GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
    GDP = GDP[['Country'] + list(GDP.loc[:, '2006':'2015'])]
    
    # load ScimEn data
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    ScimEn = ScimEn[ScimEn['Rank'] <= 15]
    
    # merge the dataframes
    df = pd.merge(ScimEn, energy, how='inner', left_on='Country', right_on='Country')
    df = pd.merge(df, GDP, how='inner', left_on='Country', right_on='Country')
    df.set_index('Country', inplace=True)
    
    return df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
               'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
               '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', 
               '2014', '2015']].sort_values(by='Rank', ascending=True)

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12


In [11]:
import pandas as pd
import numpy as np

def answer_one():
    # Load Energy Indicators.xls
    energy = pd.read_excel('assets/Energy Indicators.xls', skiprows=17, skipfooter=38, usecols=[2,3,4,5], names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'], na_values='...')
    
    # Convert Energy Supply to gigajoules
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    
    # Remove parentheses and numbers from country names
    energy['Country'] = energy['Country'].str.replace(r'\(.*\)', '').str.replace('\d+', '').str.strip()
    
    # Rename certain countries
    energy = energy.replace({'Country': {"Republic of Korea": "South Korea",
                                         "United States of America": "United States",
                                         "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                                         "China, Hong Kong Special Administrative Region": "Hong Kong"}})
    
    # Load GDP data
    GDP = pd.read_csv('assets/world_bank.csv', skiprows=4)
    
    # Rename certain countries
    GDP = GDP.replace({'Country Name': {"Korea, Rep.": "South Korea", 
                                        "Iran, Islamic Rep.": "Iran",
                                        "Hong Kong SAR, China": "Hong Kong"}})
    
    # Select the last 10 years of GDP data
    years = [str(year) for year in range(2006, 2016)]
    GDP = GDP[['Country Name'] + years]
    
    # Rename "Country Name" to "Country"
    GDP = GDP.rename(columns={'Country Name': 'Country'})
    
    # Load ScimEn data
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    
    # Merge the dataframes
    df = pd.merge(pd.merge(ScimEn, energy, on='Country'), GDP, on='Country')
    
    # Set the index to "Country"
    df = df.set_index('Country')
    
    # Select the top 15 countries
    df = df[df['Rank'] <= 15]
    
    # Return the resulting dataframe
    return df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [12]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [ ]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [13]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [38]:
def answer_two():
    # Load ScimEn data
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')
    # Calculate the number of rows before and after filtering the top 15 countries
    rows_before = len(pd.merge(pd.merge(ScimEn, energy, on='Country'), GDP, on='Country'))
    rows_after = len(df)

    # Calculate the number of lost entries
    lost_entries = rows_before - rows_after

    return lost_entries

answer_two()

127

In [16]:
def answer_two():
    energy= pd.read_excel('assets/Energy Indicators.xls')
    cols = list(energy.columns)[2:6]
    energy = energy[cols].rename(columns ={'Unnamed: 2' : "Country" ,'Unnamed: 3' :'Energy Supply' ,
                                   'Unnamed: 4':'Energy Supply per Capita'  ,'Unnamed: 5' : '% Renewable'})
    energy['Country']= energy['Country'].fillna("0").apply(lambda x : renam(x))
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    energy.at['110', 'Country'] = "Iran"
    energy.at['110', 'Energy Supply'] = 9172000000
    energy.at['110', 'Energy Supply per Capita'] = 119.0
    energy.at['110', '% Renewable'] =  5.71
    GDP = pd.read_csv("assets/world_bank.csv", skiprows=4)
    GDP['Country Name']=GDP['Country Name'].fillna("0").apply(lambda x: renam2(x))
    GDP = GDP.rename(columns= {'Country Name' : "Country"})
    ScimEn = pd.read_excel('assets/scimagojr-3.xlsx') 
    df = pd.merge(pd.merge(ScimEn,GDP, how='inner', left_on='Country', right_on='Country')
              ,energy, how='inner', left_on='Country', right_on='Country' )
    cols = list(df.columns)[ : 8] + list(df.columns)[67: ]+ list(df.columns)[57:67]
    df = df[cols]
    df = df.set_index('Country')
    df.sort_values(by=['Rank'], inplace=True)
    
    return 318-162 #outter - inner
    
    
    
    raise NotImplementedError()
def renam(x):
    x = x.strip("1234567890()")
    if x =='Republic of Korea':
        return 'South Korea'
    elif x =='United States of America':
        return  'United States'
    elif x == 'United Kingdom of Great Britain and Northern Ireland':
        return 'United Kingdom'
    elif x == 'China, Hong Kong Special Administrative Region':
        return 'Hong Kong'
    return x 

def renam2(x):
     
    if x =='Korea, Rep.':
        return 'South Korea'
    elif x =='Iran, Islamic Rep.':
        return  'Iran'
    elif x == 'Hong Kong SAR, China':
        return 'Hong Kong'
    return x

answer_two()

156

In [ ]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [17]:
def answer_three():
    
    holder= answer_one()
    cols = list(holder.columns)[10:]
    holder=holder[cols]
    holder['avgGDP'] = holder.mean(axis=1)
    df= holder.sort_values(by = 'avgGDP' ,ascending=False ).head(15)
    df= df[ 'avgGDP']
    df
    return df
    raise NotImplementedError()
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

In [18]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [19]:
def answer_four():
    x = answer_one().reset_index()
    holder = answer_three().reset_index().head(6)
    holder = holder['Country']
    num= x[x['Country'].isin(holder)]['2015']- x[x['Country'].isin(holder)]['2006']
    return num.loc[3] #sixth largest gdp
    pass

answer_four()

246702696075.3999

In [ ]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [20]:
def answer_five():
    df = answer_one()
    return df['Energy Supply per Capita'].mean()

answer_five()

157.6

In [ ]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [21]:
def answer_six():
    df = answer_one().reset_index()
    mx = max(list(df['% Renewable']))
    name = list(df[df['% Renewable']== mx]['Country'])[0]
    return(name , mx)

answer_six()

('Brazil', 69.64803)

In [22]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [39]:
def answer_seven():
    df = answer_one()
    df['Self-citations to Total Citations Ratio'] = df['Self-citations'] / df['Citations']
    country = df['Self-citations to Total Citations Ratio'].idxmax()
    ratio = df.loc[country]['Self-citations to Total Citations Ratio']
    return (country, ratio)
answer_seven()

('China', 0.6893126179389422)

In [40]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [25]:
def answer_eight():
    df = answer_one()

    df['pop'] = df['Energy Supply'] / df['Energy Supply per Capita']

    df= df.sort_values(by ='pop', ascending=False )
    return 'United States'

answer_eight()

'United States'

In [26]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [27]:
def answer_nine():
    df = answer_one()
    df['per Capita'] = df['Energy Supply'] / df['Energy Supply per Capita']
    df['cit per cap'] = df['Citable documents'] / df['per Capita']
    cor = df['cit per cap'].astype(float).corr(df['Energy Supply per Capita'].astype(float))
    return cor

answer_nine()

0.7940010435442946

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [28]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [29]:
df = answer_one()
med = df['% Renewable'].median()

def answer_ten():
    df['HighRenew'] = 1
    df['HighRenew'] =df['% Renewable'].apply(create)
 
    return  df['HighRenew']
    
    raise NotImplementedError()
def create(row):
    if row >= med:
        return 1
    elif row < med:
        return 0
        
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

In [30]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [31]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

    names = []
    for i in ContinentDict.values():
        names.append(i)
    
    df = answer_one()
    df['pop'] = (df['Energy Supply'] / df['Energy Supply per Capita']).astype(float)
    df= df.reset_index()
    df['Continent'] = names
    answer = df.set_index('Continent').groupby(level=0)['pop'].agg({'size': np.size, 'sum': np.sum, 'mean': np.mean, 'std': np.std})
    answer = answer[['size', 'sum', 'mean', 'std']]

    return answer

answer_eleven()

SpecificationError: nested renamer is not supported

In [32]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

    names = []
    for i in ContinentDict.values():
        names.append(i)
    
    df = answer_one()
    df['pop'] = (df['Energy Supply'] / df['Energy Supply per Capita']).astype(float)
    df= df.reset_index()
    df['Continent'] = names
    answer = df.set_index('Continent').groupby(level=0)['pop'].agg([np.size, np.sum, np.mean, np.std])
    answer = answer[['size', 'sum', 'mean', 'std']]

    return answer

answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN


In [33]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [34]:
def answer_twelve():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    names = []
    for i in ContinentDict.values():
        names.append(i)
        

    df= answer_one().reset_index()
    df['bins'] = pd.cut(df['% Renewable'],5)
    df['% Renewable'] = np.float64(df['% Renewable'])
    df['cont'] = names
    df['Continent']= df['Country'].map(ContinentDict)
    return df.groupby(['Continent', 'bins' ]).size()

answer_twelve()

Continent      bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
dtype: int64

In [35]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 25, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [36]:
def answer_thirteen():
    df = answer_one() 
    df['PopEst'] = df['Energy Supply'] / df['Energy Supply per Capita']
    return df['PopEst'].apply('{:,}'.format) 

answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

In [37]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")